# Análises estatísticas

A partir de várias análises estatísticas iremos entender melhor os dados assim como gerar possíveis insights.

# 1. Conhecendo os dados

Iremos conhecer os atributos dos dados assim como outras estatísticas básicas.

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import f_oneway
from scipy.stats import boxcox
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [ ]:
#Setando diretorio Data como o atual
os.chdir('../Data')

#Leitura dos dados
df_parkinson = pd.read_csv('parkinson_v1.csv',index_col='name')

#Apresentacao dos dados
df_parkinson.head()

Temos um total de 22 pessoas com Doença de Parkinson (PD) sendo que para cada uma houve duas coletas, que foram divididas em 2 experimentos com intervalo de 15 dias entre cada um. O experimento foi realizado de maneira cruzada, isso significa que em algum momento a pessoa tomou o placebo e em outro o Canabidiol (CBD).

Para cada experimento coletaram-se 7 amostras ao longo do tempo na qual extraiu-se 8 atributos, que são:

- pse: Entropia do Espectro de Potência.
- pspf1: Frequência de Pico do Espectro de Potência
- pspf2: Frequência de Pico do Espectro de Potência
- pspf3: Frequência de Pico do Espectro de Potência
- psp1: Pico do Espectro de Potência
- psp2: Pico do Espectro de Potência
- psp3: Pico do Espectro de Potência
- wspf: Pico de Frequência Ponderada do Espectro de Potência

Além desses atributos temos as seguintes informações nos dados:

- name: Nome da pessoa.
- measure: Medida realizada ao longo do tempo.
    - B: Basal
    - P: Pré-estresse
    - A: Antecipatória
    - S1: Início discurso
    - S2: Fim discurso
    - F1: Pós-estresse 1
    - F2: Pós-estresse 2
- evaluate: Indicação do número da avaliação (1 ou 2).
- drug: Droga sendo administrada (placebo ou CBD).

Observação:
- wspf = psp1 * pspf1

In [ ]:
#Quantidade de amostras e atributos
df_parkinson.shape

In [ ]:
#Quantidade de pessoas
len(df_parkinson.index.unique())

In [ ]:
#Estatísticas básicas
df_parkinson.drop(columns=['measure','evaluate','drug']).describe()

Podemos ver que temos um total de 308 amostras que foram coletadas para um total de 22 pessoas. Lembrando que cada pessoa gerou 16 amostras (8 do placebo e 8 do CBD) ao longo do tempo.

Tirando o atributo pse, os demais aparentam ter valores muito grande a partir do percentil 75, isso nos chama a atenção para possíveis outliers.

# 2. Análises


Análise estatística dos dados.

## 2.1 Análise dos dados divididos por tipo de droga


Análise estatística na qual iremos dividir os dados em dois grupos que são as classes das drogas. O propósito disso é identificar se há diferenças signiticativas para os grupos de pessoas com PD após o uso ou não do CBD.

In [ ]:
'''Plota varios boxplot definido por x e cols'''
def boxplot(df,cols,x):
    
    #Iremos plotar 3 graficos por linhas
    ncol = 3
    
    #Numero de linhas é calculado automaticamente
    nrow = int(len(cols)/3)+1
    
    fig = plt.figure(figsize=(15,9))
    fig.subplots_adjust(wspace=0.3,hspace=0.3)
    for index,c in enumerate(cols):
        ax = fig.add_subplot(nrow,ncol,index+1)
        ax.xaxis.label.set_size(15)
        ax.yaxis.label.set_size(15)
        sns.boxplot(data=df,x=x,y=c)
    plt.show()

'''Plota um pairplot customizado'''
def pairplot(df,col_plot,hue):
    
    #Pairplot
    g = sns.pairplot(data=df,hue=hue,vars=col_plot)

    #Quantidade de colunas para plotar
    n = len(col_plot)
    
    #Customizando figura
    for index in range(n):
        
        #Muda o tamanho de xticklabels 
        for tick_x in g.axes[n-1][index].get_xticklabels():
            tick_x.set_fontsize(13)

        #Muda o tamanho de yticklabels
        for tick_y in g.axes[index][0].get_yticklabels():
            tick_y.set_fontsize(13)

        #Muda o tamanho de xlabel/ylabel
        g.axes[n-1][index].xaxis.label.set_size(15)
        g.axes[index][0].yaxis.label.set_size(15)

    #Muda tamanho das legendas
    g.fig.get_children()[-1].properties()['title'].set_fontsize(15)
    g.fig.get_children()[-1].prop.set_size(15)
    plt.show()

In [ ]:
#Colunas numericas
numeric_cols = df_parkinson.columns[1:-3].values

#Pairplot
pairplot(df_parkinson,numeric_cols,'drug')

Dados que as distribuições aparentam ser assimétricas, iremos aplicar a box cox com o intuito de normalizar os dados.

In [ ]:
#Normalização via boxcox
for col in numeric_cols:
    df_parkinson.loc[:,col],_ = boxcox(df_parkinson.loc[:,col])
    
#Normalizacao dos dados - media 0 e desvio padrao 1
# df_parkinson.loc[:,numeric_cols] = StandardScaler().fit_transform(df_parkinson.loc[:,numeric_cols])

#Normalizacao dos dados - min 0 e max 1
df_parkinson.loc[:,numeric_cols] = MinMaxScaler().fit_transform(df_parkinson.loc[:,numeric_cols])

In [ ]:
#Pairplot
pairplot(df_parkinson,numeric_cols,'drug')

In [ ]:
#Boxplot divido por droga
boxplot(df_parkinson,numeric_cols,'drug')

Pelo boxplot aparentemente não há nenhuma diferença significativa nos grupos, mas há outliers.

E segundo o pairplot existe algumas relações lineares como é o caso dos psp's e pse com outras variáveis.

Além disso, em todos os atributos os espaços aparentam estar com as clases sobrepostas. Usar um classificador nesses dados provavelmente não produzirá bons resultados.

In [ ]:
#Divide dataset por droga
df_drug1 = df_parkinson[df_parkinson['drug'] == 'placebo'].copy()
df_drug2 = df_parkinson[df_parkinson['drug'] == 'CBD'].copy()

In [ ]:
#Heatmap das correlações
fig = plt.figure(figsize=(16,5))
ax = fig.add_subplot(131,title='Placebo')
sns.heatmap(round(df_drug1.loc[:,numeric_cols].corr(),2),vmin=-1,vmax=1,annot=True,ax=ax)
ax = fig.add_subplot(132,title='CBD')
sns.heatmap(round(df_drug2.loc[:,numeric_cols].corr(),2),vmin=-1,vmax=1,annot=True,ax=ax)
ax = fig.add_subplot(133,title='Tudo')
sns.heatmap(round(df_parkinson.loc[:,numeric_cols].corr(),2),vmin=-1,vmax=1,annot=True,ax=ax)
plt.show()

Olhando o heatmap temos que as principais correlações são:
- psp's entre si (Positiva).
- wspf com todos os psp's (Positiva).
- wpsf com pse (Negativa).
- psp1 com pse (Negativa)

In [ ]:
#ANOVA entre os atributos
_, pvalue = f_oneway(df_drug1.loc[:,numeric_cols],df_drug2.loc[:,numeric_cols])
dict(zip(numeric_cols,pvalue))

Segundo o anova temos difenças significativas em psp2 e psp3.

## 2.2 Análise dos dados dividos por tipo de droga porém sumarizados ao longo do tempo

Iremos realizar as análises nos mesmos grupos propostos anteriomente, contudo, faremos uma sumarização dos dados temporais para cada pessoa no seu respectivo grupo.

In [ ]:
#Média de cada atributo por droga e pessoa - sumarização dos dados temporais
df_summary_parkinson = df_parkinson.drop(columns='evaluate').groupby(['drug','name']).agg(np.mean).reset_index()

In [ ]:
#Boxplot divido por grupo para as medias dos atributos de cada pessoa
boxplot(df_summary_parkinson,numeric_cols,'drug')

In [ ]:
#Divindo dataset por tipo de droga
df_drug1 = df_summary_parkinson[df_summary_parkinson['drug'] == 'placebo'].copy()
df_drug2 = df_summary_parkinson[df_summary_parkinson['drug'] == 'CBD'].copy()

#ANOVA entre os atributos
_, pvalue = f_oneway(df_drug1.loc[:,numeric_cols],df_drug2.loc[:,numeric_cols])
dict(zip(numeric_cols,pvalue))

Realizando a média para as séries temporais temos que os boxplot's tendem a ficar mais 'shifitados'. Contudo, isso possívelmente ocorreu por causa da sumarização dos outliers, que tenderam a sumir por causa da média.

Segundo o ANOVA temos diferença apenas no atributo psp3.

In [ ]:
#Pairplot
pairplot(df_summary_parkinson,numeric_cols,'drug')

In [ ]:
#Heatmap
fig = plt.figure(figsize=(16,7))
ax = fig.add_subplot(121)
ax = sns.heatmap(df_drug1.loc[:,numeric_cols].corr(),vmin=-1,vmax=1,annot=True,ax=ax)
ax = fig.add_subplot(122)
sns.heatmap(df_drug2.loc[:,numeric_cols].corr(),vmin=-1,vmax=1,annot=True,ax=ax)
plt.show()

É interessante notar que a sumarização deixou mais visível que as classes estão distribuídas de maneira 'aleatória' para cada atributo no espaço. 

As correlações ficaram praticamente as mesmas com excecão de uma adição:
- pse com pspf1 (Positiva).

In [ ]:
#Desvio padrão de cada atributo por droga e pessoa - sumarização dos dados temporais
df_summary_parkinson = df_parkinson.drop(columns=['measure','evaluate']).groupby(['drug','name']).agg(np.std).reset_index()

In [ ]:
#Boxplot divido por grupo para o desvio padrão dos atributos de cada pessoa
boxplot(df_summary_parkinson,numeric_cols,'drug')

In [ ]:
#Divindo dataset por tipo de droga
df_drug1 = df_summary_parkinson[df_summary_parkinson['drug'] == 'placebo'].copy()
df_drug2 = df_summary_parkinson[df_summary_parkinson['drug'] == 'CBD'].copy()

#ANOVA entre os atributos
_, pvalue = f_oneway(df_drug1.loc[:,numeric_cols],df_drug2.loc[:,numeric_cols])
dict(zip(numeric_cols,pvalue))

In [ ]:
#Pairplot
pairplot(df_summary_parkinson,numeric_cols,'drug')

Segundo o ANOVA não há diferenças significativas apenas em pse. Nos chama a atenção novamente que os boxplot's estão mais 'shifitados' assim como os espaços das variáveis parecem ser mais divisíveis.

Possívelmente uma combinação da média e desvio padrão dos atributos pode ser útil.

## 2.3 Análise dos dados divididos por tipo de droga e com métricas resumidas por amostra

Iremos utilizar o mesmo agrupamento anterior mas agora com as métricas resumidas ao longo das amostras com o mesmo objetivo de avaliar diferenças significativas.

In [ ]:
#Copia
df_summary_parkinson = df_parkinson.copy()

#Excluindo colunas desnecessarias
df_summary_parkinson.drop(columns=['measure','evaluate','pse','pspf1','pspf2','pspf3','psp1','psp2','psp3'],inplace=True)

#Calculando resumos por amostras
df_summary_parkinson['pspf_mean'] = df_parkinson.apply(lambda x: x[['pspf1','pspf2','pspf3']].mean(),axis=1)
df_summary_parkinson['pspf_std'] = df_parkinson.apply(lambda x: x[['pspf1','pspf2','pspf3']].std(),axis=1)
df_summary_parkinson['psp_mean'] = df_parkinson.apply(lambda x: x[['psp1','psp2','psp3']].mean(),axis=1)
df_summary_parkinson['psp_std'] = df_parkinson.apply(lambda x: x[['psp1','psp2','psp3']].std(),axis=1)

In [ ]:
#Colunas numericas
numeric_cols = df_summary_parkinson.select_dtypes('float').columns.values

In [ ]:
#Boxplot
boxplot(df_summary_parkinson,numeric_cols,'drug')

#Divindo dataset por tipo de droga
df_drug1 = df_summary_parkinson[df_summary_parkinson['drug'] == 'placebo'].copy()
df_drug2 = df_summary_parkinson[df_summary_parkinson['drug'] == 'CBD'].copy()

#ANOVA entre os atributos
_, pvalue = f_oneway(df_drug1.loc[:,numeric_cols],df_drug2.loc[:,numeric_cols])
dict(zip(numeric_cols,pvalue))

In [ ]:
#Pairplot
pairplot(df_summary_parkinson,numeric_cols,'drug')

Como podemos ver a distribuição no espaço ainda é aleatória, contudo, foi possível obter uma signifiância no atributo pse_mean.

## 2.4 Análise dos dados dividos por tipo de droga porém sumarizados ao longo do tempo e por amostra

Iremos adicionar as outras análises o fato de que para cada amostra iremos sumarizar os dados, por exemplo, para uma amostra iremos calcular a média de psp1,psp2 e psp3. Depois disso realizaremos novamente um resumo ao longo do tempo e depois novas análises para os grupos dividos por tipo de droga.

In [ ]:
#Copia do dataframe sem as colunas que seram resumidas
df_summary_parkinson = df_parkinson.drop(columns=['evaluate','measure','pspf1','pspf2','pspf3','psp1','psp2','psp3']).copy()

#Resumo das metricas por meio da media
df_summary_parkinson['pspf'] = df_parkinson.apply(lambda x: x[['pspf1','pspf2','pspf3']].mean(),axis=1)
df_summary_parkinson['psp'] = df_parkinson.apply(lambda x: x[['psp1','psp2','psp3']].mean(),axis=1)

#Resumo das metricas ao longo do tempo
df_summary_parkinson = df_summary_parkinson.groupby(['drug','name']).agg(np.mean).reset_index()

#Colunas numericas
numeric_cols = df_summary_parkinson.columns[2:].values

In [ ]:
#Boxplot
boxplot(df_summary_parkinson,numeric_cols,'drug')

In [ ]:
#Pairplot
pairplot(df_summary_parkinson,numeric_cols,'drug')

Praticamente as mesmas conclusões obtidas pela seção anterior.

## 2.5 Análise dos dados divididos por tipo de avaliação

Iremos agora dividir os dados por meio da avaliação com o objetivo de verificar se dependendo dela os resultados da droga são diferentes.

In [ ]:
#Colunas numericas
numeric_cols = df_parkinson.columns[2:-2].values

#Divindo dados
df_evaluate1 = df_parkinson[df_parkinson['evaluate'] == 1]
df_evaluate2 = df_parkinson[df_parkinson['evaluate'] == 2]

In [ ]:
#Boxplot
boxplot(df_evaluate1,numeric_cols,'drug')

In [ ]:
#Boxplot
boxplot(df_evaluate2,numeric_cols,'drug')

In [ ]:
#ANOVA entre os atributos
_, pvalue = f_oneway(df_evaluate1.loc[df_evaluate1['drug'] == 'placebo',numeric_cols],df_evaluate1.loc[df_evaluate1['drug'] == 'CBD',numeric_cols])
dict(zip(numeric_cols,pvalue))

In [ ]:
#ANOVA entre os atributos
_, pvalue = f_oneway(df_evaluate2.loc[df_evaluate2['drug'] == 'placebo',numeric_cols],df_evaluate2.loc[df_evaluate2['drug'] == 'CBD',numeric_cols])
dict(zip(numeric_cols,pvalue))

É interessante notar que para o atributo psp3 as interpreções são diferentes dependendo da avaliação. Além disso, foi interessante constatar que os p-valores diferentes de uma avaliação para outra. 

Possívelmente essas diferenças ocorrem porque após o primeiro teste as pessoas já estão mais acostumadas.

## 2.6 Análise dos dados divididos por tipo de medida

Iremos realizar análises estatísticas nos dados separados por medida no intuito de averiguar se para diferentes instâncias de tempo temos diferenças significativas.

### 2.6.1 Boxplot's

In [ ]:
#Dividindo dataset por medida
df_measureB = df_parkinson[df_parkinson['measure'] == 'B'].drop(columns=['measure','evaluate'])
df_measureP = df_parkinson[df_parkinson['measure'] == 'P'].drop(columns=['measure','evaluate'])
df_measureA = df_parkinson[df_parkinson['measure'] == 'A'].drop(columns=['measure','evaluate'])
df_measureS1 = df_parkinson[df_parkinson['measure'] == 'S1'].drop(columns=['measure','evaluate'])
df_measureS2 = df_parkinson[df_parkinson['measure'] == 'S2'].drop(columns=['measure','evaluate'])
df_measureF1 = df_parkinson[df_parkinson['measure'] == 'F1'].drop(columns=['measure','evaluate'])
df_measureF2 = df_parkinson[df_parkinson['measure'] == 'F2'].drop(columns=['measure','evaluate'])

#Colunas numericas
numeric_cols = df_measureB.columns.values[:-1]

In [ ]:
#Boxplot - medida B
boxplot(df_measureB,numeric_cols,'drug')

#ANOVA
_,pvalue = f_oneway(df_measureB.loc[df_measureB['drug'] == 'placebo',numeric_cols],df_measureB.loc[df_measureB['drug'] == 'CBD',numeric_cols])
print(dict(zip(numeric_cols,pvalue)))

In [ ]:
#Boxplot - medida P
boxplot(df_measureP,numeric_cols,'drug')

#ANOVA
_,pvalue = f_oneway(df_measureP.loc[df_measureP['drug'] == 'placebo',numeric_cols],df_measureP.loc[df_measureP['drug'] == 'CBD',numeric_cols])
print(dict(zip(numeric_cols,pvalue)))

In [ ]:
#Boxplot - medida A
boxplot(df_measureA,numeric_cols,'drug')

#ANOVA
_,pvalue = f_oneway(df_measureA.loc[df_measureA['drug'] == 'placebo',numeric_cols],df_measureA.loc[df_measureA['drug'] == 'CBD',numeric_cols])
print(dict(zip(numeric_cols,pvalue)))

In [ ]:
#Boxplot - medida S1
boxplot(df_measureS1,numeric_cols,'drug')

#ANOVA
_,pvalue = f_oneway(df_measureS1.loc[df_measureS1['drug'] == 'placebo',numeric_cols],df_measureS1.loc[df_measureS1['drug'] == 'CBD',numeric_cols])
print(dict(zip(numeric_cols,pvalue)))

In [ ]:
#Boxplot - medida S2
boxplot(df_measureS2,numeric_cols,'drug')

#ANOVA
_,pvalue = f_oneway(df_measureS2.loc[df_measureS2['drug'] == 'placebo',numeric_cols],df_measureS2.loc[df_measureS2['drug'] == 'CBD',numeric_cols])
print(dict(zip(numeric_cols,pvalue)))

In [ ]:
#Boxplot - medida F1
boxplot(df_measureF1,numeric_cols,'drug')

#ANOVA
_,pvalue = f_oneway(df_measureF1.loc[df_measureF1['drug'] == 'placebo',numeric_cols],df_measureF1.loc[df_measureF1['drug'] == 'CBD',numeric_cols])
print(dict(zip(numeric_cols,pvalue)))

In [ ]:
#Boxplot - medida F2
boxplot(df_measureF2,numeric_cols,'drug')

#ANOVA
_,pvalue = f_oneway(df_measureF2.loc[df_measureF2['drug'] == 'placebo',numeric_cols],df_measureF2.loc[df_measureF2['drug'] == 'CBD',numeric_cols])
print(dict(zip(numeric_cols,pvalue)))

Note que não há diferenças muito grandes entre os grupos para nenhuma das fases, apesar de em alguns boxplot's termos um shift maior comparado com todos os dados juntos. Nos chama a atenção que dependendo da fase a mediana dos atributos tendem a ter 'shifts' diferentes entre os grupos. Por exemplo, o pspf3 na fase F1 é diferente do da fase F2.

### 2.6.2 Pairplot

In [ ]:
#Pairplot - medida B
pairplot(df_measureB,numeric_cols,'drug')

In [ ]:
#Pairplot - medida P
pairplot(df_measureP,numeric_cols,'drug')

In [ ]:
#Pairplot - medida A
pairplot(df_measureA,numeric_cols,'drug')

In [ ]:
#Pairplot - medida S1
pairplot(df_measureS1,numeric_cols,'drug')

In [ ]:
#Pairplot - medida S2
pairplot(df_measureS2,numeric_cols,'drug')

In [ ]:
#Pairplot - medida F1
pairplot(df_measureF1,numeric_cols,'drug')

In [ ]:
#Pairplot - medida F2
pairplot(df_measureF2,numeric_cols,'drug')

Pelos pairplot's não é possível verificar nenhum espaço que seja muito separável.

## 2.7 Análise dos dados divididos por tipo de medida e com métricas resumidas por amostra

Iremos utilizar o mesmo agrupamento anterior mas agora com as métricas resumidas ao longo das amostras com o mesmo objetivo de avaliar diferenças significativas.

In [ ]:
#Dividindo dataset por medida
df_measureB = df_parkinson[df_parkinson['measure'] == 'B'].drop(columns=['measure','evaluate'])
df_measureP = df_parkinson[df_parkinson['measure'] == 'P'].drop(columns=['measure','evaluate'])
df_measureA = df_parkinson[df_parkinson['measure'] == 'A'].drop(columns=['measure','evaluate'])
df_measureS1 = df_parkinson[df_parkinson['measure'] == 'S1'].drop(columns=['measure','evaluate'])
df_measureS2 = df_parkinson[df_parkinson['measure'] == 'S2'].drop(columns=['measure','evaluate'])
df_measureF1 = df_parkinson[df_parkinson['measure'] == 'F1'].drop(columns=['measure','evaluate'])
df_measureF2 = df_parkinson[df_parkinson['measure'] == 'F2'].drop(columns=['measure','evaluate'])

In [ ]:
#Calculando resumos para cada dataframe
for df in [df_measureB,df_measureP,df_measureA,df_measureS1,df_measureS2,df_measureF1,df_measureF2]:
    df['pspf_mean'] = df.apply(lambda x: x[['pspf1','pspf2','pspf3']].mean(),axis=1)
    df['pspf_std'] = df.apply(lambda x: x[['pspf1','pspf2','pspf3']].std(),axis=1)
    df['psp_mean'] = df.apply(lambda x: x[['psp1','psp2','psp3']].mean(),axis=1)
    df['psp_std'] = df.apply(lambda x: x[['psp1','psp2','psp3']].std(),axis=1)

    #Excluindo colunas desnecessarias
    df.drop(columns=['pspf1','pspf2','pspf3','psp1','psp2','psp3'],inplace=True)

In [ ]:
#Colunas numericas
numeric_cols = df_measureB.drop(columns='drug').columns.values

#Boxplot - medida B
boxplot(df_measureB,numeric_cols,'drug')

#ANOVA
_,pvalue = f_oneway(df_measureB.loc[df_measureB['drug'] == 'placebo',numeric_cols],df_measureB.loc[df_measureB['drug'] == 'CBD',numeric_cols])
print(dict(zip(numeric_cols,pvalue)))

In [ ]:
#Boxplot - medida P
boxplot(df_measureP,numeric_cols,'drug')

#ANOVA
_,pvalue = f_oneway(df_measureP.loc[df_measureP['drug'] == 'placebo',numeric_cols],df_measureP.loc[df_measureP['drug'] == 'CBD',numeric_cols])
print(dict(zip(numeric_cols,pvalue)))

In [ ]:
#Boxplot - medida A
boxplot(df_measureA,numeric_cols,'drug')

#ANOVA
_,pvalue = f_oneway(df_measureA.loc[df_measureA['drug'] == 'placebo',numeric_cols],df_measureA.loc[df_measureA['drug'] == 'CBD',numeric_cols])
print(dict(zip(numeric_cols,pvalue)))

In [ ]:
#Boxplot - medida S1
boxplot(df_measureS1,numeric_cols,'drug')

#ANOVA
_,pvalue = f_oneway(df_measureS1.loc[df_measureS1['drug'] == 'placebo',numeric_cols],df_measureS1.loc[df_measureS1['drug'] == 'CBD',numeric_cols])
print(dict(zip(numeric_cols,pvalue)))

In [ ]:
#Boxplot - medida S2
boxplot(df_measureS2,numeric_cols,'drug')

#ANOVA
_,pvalue = f_oneway(df_measureS2.loc[df_measureS2['drug'] == 'placebo',numeric_cols],df_measureS2.loc[df_measureS2['drug'] == 'CBD',numeric_cols])
print(dict(zip(numeric_cols,pvalue)))

In [ ]:
#Boxplot - medida F1
boxplot(df_measureF1,numeric_cols,'drug')

#ANOVA
_,pvalue = f_oneway(df_measureF1.loc[df_measureF1['drug'] == 'placebo',numeric_cols],df_measureF1.loc[df_measureF1['drug'] == 'CBD',numeric_cols])
print(dict(zip(numeric_cols,pvalue)))

In [ ]:
#Boxplot - medida F2
boxplot(df_measureF2,numeric_cols,'drug')

#ANOVA
_,pvalue = f_oneway(df_measureF2.loc[df_measureF2['drug'] == 'placebo',numeric_cols],df_measureF2.loc[df_measureF2['drug'] == 'CBD',numeric_cols])
print(dict(zip(numeric_cols,pvalue)))

Mesmas conclusões para as medidas calculadas no tópico 2.6

# 3. Conclusões

- Sem nenhum tratamento nos dados provavelmente não será possível usar nenhum algoritmo de machine learning.
- Na forma como eles estão, não há diferenças significativas entre os grupos divididos por tipo de droga.
- Usar sumarização nos dados temporais pode ser uma boa ideia como médio e desvio padrão em conjunto por apararentemente gerar espaços mais separaveis.
- Lembrando que ao realizar resumos nas métricas estaremos diminuindo o número de amostras.
- Realizar resumos por amostra não parece ser interessante por não ter produzido resultados interessantes.
- Dependendo da fase as conclusões para um mesmo atributo pode ser diferente pois quando voce analisa os boxplot's temos que 'shift's diferentes. Além disso, temos que há fases na qual as diferenças aparenta ser mais significativa. Portanto, pode ser interessante gerar variáveis dummies que indiquem qual o tipo de fase a pessoa se encontra.